.model .pt converter

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from miditok import REMI
from pathlib import Path
import time

# LSMT with Attention
from LSTMwithAtt import LSTMwithAtt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = REMI(params=Path("tokenizers/piano1_token/tokenizer.json"))

hidden_size = 200
model = LSTMwithAtt(tokenizer, hidden_size)

model.load_state_dict(torch.load('models/lstmwithatt_better.model',map_location=torch.device('cpu')))
'''
outfile = "models/" + "lstmwithatt_best.pt"
torch.save(model.state_dict(),outfile)  
'''

/Users/hapticslab/Programming/humusic/.venv/lib/python3.9/site-packages/miditok/tokenizations/remi.py:77: UserWarning: Attribute controls are not compatible with 'config.one_token_stream_for_programs' and multi-vocabulary tokenizers. Disabling them from the config.
  super().__init__(tokenizer_config, params)
/var/folders/7f/fjwjx3xd5mbf2t9x4r4k4__00000gn/T/ipykernel_70965/1435241412.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlist

'\noutfile = "models/" + "lstmwithatt_best.pt"\ntorch.save(model.state_dict(),outfile)  \n'

In [8]:
# 入力トークン列の例
prompt = ("prompts/piano1_only_bach_850_0.mid")

input_ids = torch.tensor([tokenizer(prompt)]).to(device)
input_ids.shape, input_ids

(torch.Size([1, 490]),
 tensor([[ 2973,  3465,    58,  8344,  2729, 13485,  3300, 11913,  1503,  2650,
          18858,  4918,  3197, 20006,  3091,  1458,  6462,  3850,  2259, 22554,
           2650,  1845,  2873,  1235,   482,  2436,  3465,    58,  8344,  4641,
           5528,  1501,  5040, 11913,  1503,  5266,  2024,  4303,  1559,  8208,
           3197, 11135,  2731,  3091,  1458,  6462,  2778,  3194,  2290,  1229,
           7308, 18991,  3786, 22734,  1502,  9196,  2108,  6167,  4255,  6899,
           8341,  3827, 22968,  3911,  2048,  5039,  2254,  7291, 23365,  5517,
           2211, 18528,  7858, 18145,  1097, 11482, 22734,  3487,    46,  4187,
           6897,  4255, 10990,  4428,  5019,  2018,    41,   382, 22968,   369,
          15713, 17062,    41,   365, 18461, 15636,   378,  8341,  7308, 28398,
           3081, 26001,    48,   339,  6526,  4104,  5514,  8386,  4668,  1906,
           4464,  3406,  6543,  5893,  2561,  1582,  3367,  9282,  4474,  2118,
           4111, 

In [11]:

#torch.set_printoptions(threshold=torch.inf)
model.eval()

gen_token = []

with torch.no_grad():
    #prompt_input = torch.LongTensor( input_ids ).to(device)
    #print(input_ids.shape)
    x = model.input_emb(input_ids)
    #print(x.shape)
    ox, (hnx, cnx) = model.lstm1(x)
    #print(ox.shape, hnx.shape,cnx.shape)
    hnx, cnx = hnx[:,0,:], cnx[:,0,:]
    
    wid = input_ids[0][0]
    
    sl = 0
    while True:
        wids = torch.LongTensor([wid]).to(device)
        y = model.answer_emb(wids)
        #print(y.shape)
        
        oy, (hnx, cnx) = model.lstm2(y, (hnx, cnx))
        oy = oy.unsqueeze(1)
        ox1 = ox.permute(0,2,1)
        sim = torch.bmm(oy,ox1)
        bs, yws, xws = sim.shape
        sim2 = sim.reshape(bs*yws,xws)
        alpha = F.softmax(sim2,dim=1).reshape(bs, yws, xws)
        ct = torch.bmm(alpha,ox)
        oy1 = torch.cat([ct,oy],dim=2)
        oy2 = model.Wc(oy1)
        oy3 = model.W(oy2)
        wid = torch.argmax(oy3[0]).item()
        gen_token.append(wid)
        #if (wid == esid):
            #break
        if (sl == 2000):
            break
        sl += 1
    print(gen_token)

[6877, 17955, 1735, 19508, 11337, 29262, 6373, 1889, 8018, 6877, 1889, 15789, 7996, 8825, 23004, 9739, 9848, 7027, 9896, 1660, 5685, 2781, 17888, 4171, 11835, 5685, 16650, 2781, 17888, 555, 11337, 7674, 12799, 13156, 17888, 3363, 11337, 15210, 17888, 17888, 17888, 23004, 5718, 17497, 17888, 4879, 8185, 20484, 25, 10102, 19377, 19508, 4879, 1889, 12032, 885, 14379, 258, 22701, 17888, 17888, 2351, 11337, 18558, 4697, 4989, 12558, 10102, 15789, 15640, 6023, 15210, 17888, 17888, 16654, 909, 4582, 4879, 1889, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 17888, 

In [13]:
generated = tokenizer.decode(gen_token)
print("Generated MIDI Tokens:", generated)
generated_time = str(time.time())

generated.dump_midi("generated/generated_test_"+ generated_time +".mid")
print("generated_test_"+ generated_time +".mid")

Generated MIDI Tokens: Score(ttype=Tick, tpq=8, begin=0, end=3170, tracks=1, notes=1848, time_sig=1, key_sig=0, markers=0)
generated_test_1734344474.4607239.mid


In [14]:
#pygameによる再生

file_name = Path("generated/generated_test_"+ generated_time +".mid")

import time
import pygame

# Initialize pygame
pygame.init()

# Set up the mixer
pygame.mixer.init()

# Load the MIDI file
#pygame.mixer.music.load('bach_850.mid')
pygame.mixer.music.load(file_name)
#pygame.mixer.music.load('test.mid')

# Play the MIDI file
pygame.mixer.music.set_volume(0.25)
pygame.mixer.music.play()


# Keep the program running until the music stops
while pygame.mixer.music.get_busy():
    if input("Press 'q' to quit: ") == 'q':
        break
    pygame.time.Clock().tick(10)

# Quit pygame
pygame.quit()

pygame 2.6.1 (SDL 2.28.4, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html
